In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from keras.models import load_model
classifier_B_model = load_model('/content/drive/My Drive/NLP/models/LSTM_GLOVE_Pos_Emotions.h5')
classifier_C_model = load_model('/content/drive/My Drive/NLP/models/LSTM_GLOVE_Neg_Emotions.h5')

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
classifier_B_model.summary()

Model: "Pre-trained Glove.6B.300d"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 280, 300)          6555000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 6,597,690
Trainable params: 6,597,690
Non-trainable params: 0
_________________________________________________________________


In [ ]:
classifier_C_model.summary()

Model: "Pre-trained Glove.6B.300d"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 280, 300)          5148300   
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 99        
Total params: 5,191,023
Trainable params: 5,191,023
Non-trainable params: 0
_________________________________________________________________


In [ ]:
%tensorflow_version 1.x



from keras import backend as K
#K.tensorflow_backend._get_available_gpus()


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd 
from sklearn.metrics import precision_score , recall_score
  
def append_emotion(input_file, pos_emotion_file, neg_emotion_file):
  dataset = input_file
  data = pd.read_csv(dataset) 
  data.drop(data.loc[data.sentiment != 1].index, inplace=True)
  #prepare data
  maxlen = 280
  max_words = 40000
  tokenizer = Tokenizer(num_words=max_words)
  #tokenizer.fit_on_texts(data['Tweet'])
  #sequences = tokenizer.texts_to_sequences(data['Tweet'])

  tokenizer.fit_on_texts(data['text'])
  sequences = tokenizer.texts_to_sequences(data['text'])

  word_index = tokenizer.word_index
  print('Found %s unique tokens.' % len(word_index))
  data1 = pad_sequences(sequences, maxlen=maxlen)
  y_val_pred=classifier_B_model.predict_classes(data1)
  data['emotion'] = y_val_pred
  data.to_csv(pos_emotion_file)

  #write negative emotions
  data = pd.read_csv(dataset) 
  data.drop(data.loc[data.sentiment != 0].index, inplace=True)
  #prepare data
  maxlen = 280
  max_words = 40000
  tokenizer = Tokenizer(num_words=max_words)
  #tokenizer.fit_on_texts(data['Tweet'])
  #sequences = tokenizer.texts_to_sequences(data['Tweet'])

  tokenizer.fit_on_texts(data['text'])
  sequences = tokenizer.texts_to_sequences(data['text'])

  word_index = tokenizer.word_index
  print('Found %s unique tokens.' % len(word_index))
  data1 = pad_sequences(sequences, maxlen=maxlen)
  y_val_pred=classifier_C_model.predict_classes(data1)
  data['emotion'] = y_val_pred
  data.to_csv(neg_emotion_file)
  
  



TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
input_file = '/content/drive/My Drive/NLP/COVID/Country_Sentiments/feb_dataset/Pakistan.CSV'
pos_file = '/content/drive/My Drive/NLP/COVID/country_emotions/Pakistan_Pos_Emotions.csv'
neg_file = '/content/drive/My Drive/NLP/COVID/country_emotions/Pakistan_Neg_Emotions.csv'
append_emotion(input_file,pos_file,neg_file)

Found 5347 unique tokens.
Found 8175 unique tokens.
